In [ ]:
## Collecte de données

Vous devez collecter et télécharger un ensemble d'images. Vous avez les tâches suivantes à programmer, en automatisant le processus autant que possible :

1.  Créer un dossier appelé *images*.
2.  Télécharger les images sous licence ouverte dans le dossier *images* (minimum 100
    images).
3.  Enregistrez les métadonnées de chaque image comme la taille de l'image, le format de l'image (.jpeg,
    .png, etc.), l'orientation de l'image (paysage, portrait, carré, etc.),
    date de création, modèle d'appareil photo, etc. dans un ou plusieurs fichiers JSON. Vous pouvez utiliser les informations [Exif](https://en.wikipedia.org/wiki/Exif) présentes dans les fichiers d'images.

In [9]:
import os

# Spécifiez le chemin du dossier que vous souhaitez créer
dossier = 'images'

# Créez le dossier
os.makedirs(dossier, exist_ok=True)

print(f"Le dossier '{dossier}' a été créé avec succès.")


Le dossier 'images' a été créé avec succès.


In [5]:
!pip install SPARQLWrapper

In [10]:
import os
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor

# Initialiser le wrapper SPARQL
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# Définir la requête SPARQL pour obtenir des images de monuments
sparql.setQuery("""
SELECT ?monument ?monumentLabel ?image WHERE {
  ?monument wdt:P31 wd:Q839954.  # Monument
  ?monument wdt:P18 ?image.      # Image
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 100
""")

sparql.setReturnFormat(JSON)

try:
    # Exécuter la requête et récupérer les résultats en JSON
    results = sparql.query().convert()
except Exception as e:
    print(f"❌ Erreur lors de l'exécution de la requête SPARQL : {e}")
    exit(1)

# Liste des images à télécharger avec des indices de 1 à 100
images = [(i + 1, result["image"]["value"]) for i, result in enumerate(results["results"]["bindings"])]

# Définition d'un User-Agent personnalisé
HEADERS = {
    "User-Agent": "MonScript/1.0 (mailto:mohamedguef@gmail.com)"  # Remplace par ton email pour respecter la politique
}

def download_image(data):
    index, image_url = data
    try:
        # Nom fixe sous le format image_1.jpg, image_2.jpg, ..., image_100.jpg
        image_name = f"image_{index}.jpg"
        image_path = os.path.join(IMAGE_DIR, image_name)

        # Télécharger l'image avec un User-Agent correct
        response = requests.get(image_url, headers=HEADERS, timeout=10)
        response.raise_for_status()  # Vérifie si la requête a réussi
        
        # Sauvegarder l'image
        with open(image_path, 'wb') as f:
            f.write(response.content)
        
        print(f"✅ Téléchargé : {image_name}")

    except requests.RequestException as e:
        print(f"⚠️ Erreur lors du téléchargement de {image_url} : {e}")
    except Exception as e:
        print(f"⚠️ Erreur inattendue : {e}")

# Téléchargement des images en parallèle
MAX_THREADS = 10  # Ajuste ce nombre en fonction de ta connexion et des performances

with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    executor.map(download_image, images)

print("✅ Tous les téléchargements sont terminés !")


✅ Téléchargé : image_6.jpg
✅ Téléchargé : image_7.jpg
✅ Téléchargé : image_5.jpg
✅ Téléchargé : image_1.jpg
✅ Téléchargé : image_2.jpg
✅ Téléchargé : image_3.jpg
✅ Téléchargé : image_8.jpg
✅ Téléchargé : image_10.jpg
✅ Téléchargé : image_9.jpg
✅ Téléchargé : image_4.jpg
✅ Téléchargé : image_11.jpg
✅ Téléchargé : image_12.jpg
✅ Téléchargé : image_20.jpg
✅ Téléchargé : image_13.jpg
✅ Téléchargé : image_21.jpg
✅ Téléchargé : image_22.jpg
✅ Téléchargé : image_24.jpg
✅ Téléchargé : image_14.jpg
✅ Téléchargé : image_19.jpg
✅ Téléchargé : image_18.jpg
✅ Téléchargé : image_23.jpg
✅ Téléchargé : image_16.jpg
✅ Téléchargé : image_25.jpg
✅ Téléchargé : image_30.jpg
✅ Téléchargé : image_27.jpg
✅ Téléchargé : image_17.jpg
✅ Téléchargé : image_32.jpg
✅ Téléchargé : image_31.jpg
✅ Téléchargé : image_15.jpg
✅ Téléchargé : image_26.jpg
✅ Téléchargé : image_34.jpg
✅ Téléchargé : image_35.jpg
✅ Téléchargé : image_36.jpg
✅ Téléchargé : image_39.jpg
✅ Téléchargé : image_40.jpg
✅ Téléchargé : image_37.jpg
✅

In [14]:
!pip install Pillow ExifRead

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.9 MB/s eta 0:00:00


In [7]:
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 789.2 kB/s eta 0:00:00MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 1.1 MB/s eta 0:00:003.3 MB/s eta 0:00:01


In [17]:
import os
import json
from PIL import Image, UnidentifiedImageError
import exifread
from tqdm import tqdm  # Barre de progression

# Définir le dossier des images et le fichier de sortie JSON
IMAGE_DIR = "images"
METADATA_FILE = os.path.join(IMAGE_DIR, "metadata.json")

# Vérifier que le dossier "images" existe
if not os.path.exists(IMAGE_DIR):
    print(f"⚠️ Dossier '{IMAGE_DIR}' introuvable. Vérifiez que les images sont bien téléchargées.")
    exit(1)

# Liste pour stocker les métadonnées de toutes les images
metadata_list = []

# Liste des fichiers images triés
image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

# Parcourir les images et extraire les métadonnées
for image_name in tqdm(image_files, desc="📷 Extraction des métadonnées"):
    image_path = os.path.join(IMAGE_DIR, image_name)

    try:
        # Vérifier si l'image est corrompue en tentant de l'ouvrir
        with Image.open(image_path) as img:
            img.verify()  # Vérifie l'intégrité de l'image sans la charger
            img = Image.open(image_path)  # Recharge l'image pour la lecture des métadonnées
            width, height = img.size
            format_ = img.format
            orientation = 'Portrait' if height > width else 'Paysage' if width > height else 'Carré'

        # Vérifier si c'est un PNG (pas d'Exif)
        if format_.upper() == "PNG":
            exif_data = "Non disponible (format PNG)"
        else:
            # Extraire les métadonnées Exif avec exifread
            with open(image_path, 'rb') as f:
                tags = exifread.process_file(f, stop_tag="EXIF DateTimeOriginal", details=False)

            # Filtrer les champs problématiques
            exif_data = {
                "Date de création": str(tags.get("EXIF DateTimeOriginal", "Inconnu")),
                "Modèle d'appareil": str(tags.get("Image Model", "Inconnu")),
                "Marque d'appareil": str(tags.get("Image Make", "Inconnu")),
                "ISO": str(tags.get("EXIF ISOSpeedRatings", "Inconnu")),
                "Temps d'exposition": str(tags.get("EXIF ExposureTime", "Inconnu")),
                "Ouverture (f)": str(tags.get("EXIF FNumber", "Inconnu")),
                "Longueur focale": str(tags.get("EXIF FocalLength", "Inconnu")),
                "GPS Latitude": str(tags.get("GPS GPSLatitude", "Non disponible")),
                "GPS Longitude": str(tags.get("GPS GPSLongitude", "Non disponible")),
            }

            # Supprimer les champs corrompus (évite les erreurs "Possibly corrupted field")
            exif_data = {key: value for key, value in exif_data.items() if "Possibly corrupted" not in value}

        # Ajouter les métadonnées à la liste
        metadata_list.append({
            'Nom du fichier': image_name,
            'Taille': {'Largeur': width, 'Hauteur': height},
            'Format': format_,
            'Orientation': orientation,
            'Exif': exif_data
        })

    except UnidentifiedImageError:
        print(f"⚠️ Image illisible : {image_name} (corrompue ou format inconnu)")
    except Exception as e:
        print(f"⚠️ Erreur inattendue sur {image_name} : {e}")

# Enregistrer toutes les métadonnées dans un fichier JSON
try:
    with open(METADATA_FILE, 'w', encoding='utf-8') as f:
        json.dump(metadata_list, f, indent=4, ensure_ascii=False)
    print(f"✅ Métadonnées enregistrées dans '{METADATA_FILE}'")
except Exception as e:
    print(f"❌ Erreur lors de l'enregistrement du fichier JSON : {e}")


Possibly corrupted field ImageDescription in Image IFD  | 0/100 [00:00<?, ?it/s]
Possibly corrupted field Artist in Image IFD
Possibly corrupted field Copyright in Image IFD
📷 Extraction des métadonnées: 100%|████████| 100/100 [00:00<00:00, 2432.96it/s]

⚠️ Image illisible : image_91.jpg (corrompue ou format inconnu)
⚠️ Image illisible : image_92.jpg (corrompue ou format inconnu)
⚠️ Image illisible : image_93.jpg (corrompue ou format inconnu)
⚠️ Image illisible : image_94.jpg (corrompue ou format inconnu)
⚠️ Image illisible : image_95.jpg (corrompue ou format inconnu)
✅ Métadonnées enregistrées dans 'images/metadata.json'
